Importing libraries and initialising global constants. This code is only valid for two non identical S wave states, as such any matrix elements are not general and should not be copied mindlessly to other codes.

In [45]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.special import gamma, factorial

STARTING_RANGE_PARAMETER = 0.1 # In [fm^-2]
ENDING_RANGE_PARAMETER = 5
REDUCED_MASS = 935 * 10/11 # In [Mev / c^2], need to update value and units (10/11 A in MeV)
SUM_LIMIT = 10 # Determines the number of gaussians we expand our wave function to
CORE_MS_RADIUS = 2.30**2 # In fm^2, taken from p.232 of Tanihata et. al. (2013)

V_LS = 21.0 # In MeV
DIFFUSIVITY = 0.6 # Diffusivity, may want to check the vaidity of this paticular number
r_0 = 1.2 # In fm, may want to chose a better value for small nuclei
A_C = 10 # The number of nucleons in the core
SIGMA = 2 #fm

CENTRAL_POTENTIAL_PARAMETERS = [0.1, 0.151991, 0.231013, 0.351119, 0.53367, 0.811131, 1.23285,
                                1.87382, 2.84804, 4.32876, 6.57933, 10.]

CENTRAL_MIXING_COEFFICIENTS = [0.0558247,0.214443,2.42773,-0.724055,-2.17761,1.02031,0.819031,-0.96538,0.197094,0.3221,-0.296652,0.093208]

SPIN_ORBIT_POTENTIAL_PARAMETERS = [4.16493, 2.843, 1.941, 1.325, 0.905, 0.618, 0.422, 0.288, 0.196, 0.134, 0.0916, 0.0625]

SPIN_ORBIT_MIXING_COEFFICIENTS = [0.273, -1.307, 3.305, -5.657, 6.565, -3.368, -2.437, 2.638, 0.719, 0.235, 0.0186, 0.00108]

In [46]:
def overlap_matrix_element(range_parameter_i, range_parameter_k, range_parameter_j, range_parameter_l):
    term_1 = range_parameter_i * range_parameter_k * range_parameter_j * range_parameter_l
    term_2 = (range_parameter_i**2 + range_parameter_k**2) * (
        range_parameter_j**2 + range_parameter_l**2)
    return 8 * (term_1 / term_2)**1.5

def kinetic_matrix_element(range_parameter_i, range_parameter_k):
    term_1 = range_parameter_i * range_parameter_k
    term_2 = range_parameter_i**2 + range_parameter_k**2
    return 24 * np.sqrt(2) * term_1**(1.5) / term_2**(2.5)

def potential_matrix_element(range_parameter_i, range_parameter_k, central_potential_mixing_coefficient,
                             central_potential_param):
    V_0 = -11.405 * (-1)**0 - 51.175 # Defines V_0 for odd and even l states, shifted from values in capel et. al.
    term_1 = range_parameter_i * range_parameter_k
    term_2 = (2 / range_parameter_i**2) + (2 / range_parameter_k**2) + central_potential_param
    return 8 * central_potential_mixing_coefficient * V_0 / (term_1 * term_2)**(1.5)


In [47]:
def matrix_generation(central_mixing_coefficients=CENTRAL_MIXING_COEFFICIENTS,
                      central_potential_parameters=CENTRAL_POTENTIAL_PARAMETERS,
                      size=SUM_LIMIT):
    first_h_matrix = np.zeros(shape=(size**2, size**2))
    second_h_matrix = np.zeros(shape=(size**2, size**2))
    n_matrix = np.zeros(shape=(size**2, size**2))

    for i in range(size):
        i_range_parameter = next_range_parameter(i)
        for j in range(size):
            j_range_parameter = next_range_parameter(j)
            for k in range(size):
                k_range_parameter = next_range_parameter(k)
                for l in range(size):
                    l_range_parameter = next_range_parameter(l)
                    potential_energy_term = 0
                    I = (j - 1) * size + i
                    J = (l - 1) * size + l
                    n_matrix[I, J] = overlap_matrix_element(i_range_parameter, k_range_parameter,
                                                            j_range_parameter, l_range_parameter)
                    if j == l:
                        kinetic_term = kinetic_matrix_element(i_range_parameter, k_range_parameter)
                        for m in range(len(central_mixing_coefficients)):
                            potential_energy_term += potential_matrix_element(
                                i_range_parameter, k_range_parameter, central_mixing_coefficients[m],
                                central_potential_parameters[m])
                        first_h_matrix[I, J] = kinetic_term + potential_energy_term
                    elif i == k:
                        kinetic_term = kinetic_matrix_element(j_range_parameter, l_range_parameter)
                        for m in range(len(central_mixing_coefficients)):
                            potential_energy_term += potential_matrix_element(
                                j_range_parameter, l_range_parameter, central_mixing_coefficients[m],
                                central_potential_parameters[m])
                        second_h_matrix[I, J] = kinetic_term + potential_energy_term

    return first_h_matrix + second_h_matrix, n_matrix


def next_range_parameter(i, starting_range_parameter=STARTING_RANGE_PARAMETER, ending_range_parameter=ENDING_RANGE_PARAMETER,
                         sum_limit=SUM_LIMIT):
    """
    Finds the next range parameter given the previous and initial range parameters.
    Currently using a simple geometric series to determine range parameters.
    Chose geometric basis parameters $\alpha_i = \alpha_1a^{i-1}$ with initial parameters $\alpha_1 = 0.01, a=2$

    Parameters
    ----------
    i : int detailing the iteration number

    Returns
    -------
    new_range_parameter: float

    """
    geometric_progression_number = (ending_range_parameter / starting_range_parameter)**(1 / (sum_limit - 1))
    new_range_parameter = starting_range_parameter * geometric_progression_number**(i)

    return new_range_parameter

In [48]:
h_matrix, n_matrix = matrix_generation()
print(np.where(n_matrix<0))
s_eigenvalues, s_eigenvectors = scipy.linalg.eigh(h_matrix, n_matrix)
s_overlap_eigenvalues, s_overlap_eigenvectors = scipy.linalg.eigh(n_matrix)
s_overlap_matrix_condition_number = np.max(s_overlap_eigenvalues) / np.min(s_overlap_eigenvalues)
print(f"The s 1/2 overlap matrix condition number is", s_overlap_matrix_condition_number)

s0_eigenvector = np.asmatrix(s_eigenvectors[:, 0])
s1_eigenvector = np.asmatrix(s_eigenvectors[:, 1])
print("The S state eigenvalues are", s_eigenvalues)
print("The S1 eigenvector is", s1_eigenvector)

(array([], dtype=int64), array([], dtype=int64))


LinAlgError: The leading minor of order 1 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.